In [1]:
# setting
from operator import add
from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster('local').setAppName('test')

sc = SparkContext(conf = conf)

---------------------------------------------------------------------------------------

### additional

### 어떤 일의 순위가 있는 경우 병렬처리가 힘들다. 

### t1 -> t2 와 같이 t2 가 이루어지기 위해 t1 이 필요한 경우 병렬처리가 힘들다.

### 즉 각각의 task 들이 독립적으로 작업이 가능할 때 병렬상태이다. 라고 이해하면 좋겠다.

---------------------------------------------------------------------------------------

### Reduction Actions
### 근접하는 elements 들을 모아서 하나의 결과로 만든다.
__methods__ 와 파라미터는 다음과 같고, 코드로 하나씩 살펴보자. 

1. .reduce()

   ex - ```RDD.reduce(function)```

2. .fold()

   ex - ```RDD.fold(startvalue(aka. zerovalue), function)```

3. .groupby()

   ex - ```RDD.groupBy(function)```

4. .aggregate()

   ex - ```RDD.aggregate(startvalue, seqOp(change type), combOp(reduction function : add, lambda x,y,z: ~))```

# 1. reduce

In [5]:
rdd1 = sc.parallelize([1,2,3,4,5])

rdd1.reduce(add)

15

결과를 보면 알 수 있 듯, reduce 라는 method 는 사용자가 지정한 함수를 받아서 단일 값으로 줄여주는 역할을 한다. 

In [6]:
rdd1.reduce(lambda x, y: x + y)

15

위의 코드도 처음의 코드와 같은 역할을 하는 것을 알 수 있다. 즉 어떤 함수의 내용을 적용받아 하나의 단일 값으로 바꿔준다.

In [9]:
rdd2 = sc.parallelize(['he', 'll', 'o', '!'])

rdd2.reduce(add)

'hello!'

string type 도 가능하다는 점을 보여준다.

좀 더 여러운 버전도 해보자.

In [10]:
rdd3 = sc.parallelize([1,2,3,4], 2)

위의 코드는 RDD 를 2 개의 파티션으로 분할해서 만든다 라는 코드이다.

```sc.parallelize([dataframe, list, array], n)``` 이런 식인데 대충 느낌이 오지 않나??

In [11]:
rdd3.reduce(lambda x, y: (x*2) + y)

18

위 코드의 연산 방식을 살펴보면 파티션을 두개로 나누니까 [1,2], [3,4] 로 나뉘었을 것이고.. 

첫 번째 파티션의 결과는 1*2 + 2 = 4

두 번째 파티션의 결과는 3*2 + 4 = 10

그리고 두 개의 파티션의 결과는 4*2 + 10 = 18 이 나온 것이다. 처음에는 이해하기 힘들더라....

In [12]:
rdd4 = sc.parallelize([1,2,3,4,5], 2)

위의 코드처럼 RDD 의 elements 가 홀수인데 파티셔닝을 짝수로 or RDD 의 elements 가 짝수인데 파티셔닝을 홀수로 하면 어떻게 될까..?

In [14]:
rdd4.reduce(lambda x, y: (x*2) + y)

33

case 1. [1,2,3] , [4,5] 로 분할된 경우 .... 는 없다.

1 * 2 + 2 = 4

4 * 2 + 3 = 11

4 * 2 + 5 = 13

22 + 13 = 35 된다면 대략 이런 식으로 계산되지 않을까?

__홀짝 or 짝홀의 경우 무조건 마지막 번째의 파티션이 많은 기준으로 파티셔닝 하기 때문__ -> chatgpt 출처

case 2. [1,2] , [3,4,5] 파티셔닝한다. 

1 * 2 + 2 = 4

3 * 2 + 4 = 10

10 * 2 + 5 = 25

4 * 2 + 25 = 33

case 2. 와 같은 방식으로 연산하게 된다. 

case 1. 과 같은 방식으로는 연산되지 않는다는 점을 기억하자. 

솔직히 chatgpt 를 무한정 신뢰할 수는 없다. 근데 계산결과가 그렇다.

In [16]:
rdd5 = sc.parallelize([1,2,3,4,5], 3)

rdd5.reduce(lambda x, y: (x*2) + y)

31

[1, [2, 3] , [4, 5]] 으로 파티셔닝하고,

1    첫 번째 파티션 계산 결과    ->   __짜투리는 y 에 해당하는 값이 존재하지 않아 연산하지 않는다.__

2 * 2 + 3 = 7 두 번째 파티션 계산 결과

4 * 2 + 5 = 13 세 번째 파티션 계산 결과

1(1p) * 2 + 7(2p) = 9 

9(1p,2p 연산 결과) * 2 + 13(3p 연산 결과) = 31 

솔직히 아직도 어떻게 파티셔닝이 될 지는 잘 모르겠다. 

위에 기술한 방식도 chatgpt 에서 정보를 받아 해보기도 하고, 몇차례 혼자 이것 저것 넣어서 실험한 결과라..

chatgpt 의 답변도 일관되지 않았고, 구글링을 해도 의견이 분분했다.

그래서 개인적으로는 연산의 순서와 상관 없이, 결과값을 보장하기 위해서는 교환법칙, 결합법칙을 생각하며 코딩하는 것이 좋겠다.

__교환 법칙 = (a * b) = (b * a) 순서를 바꿔서 곱해도 값에 변화가 없다.__

__결합 법칙 = (a * b) * c = a * (b * c) 어떻게 묶어서 곱하더라도 값에 변화가 없다.__

솔직히 위의 법칙들은 초등학교 때 배우지 않나?? 저걸 여기서 생각해야 할 줄은 몰랐다.

# 2. fold

In [17]:
rdd1.fold(0, add)

15

이건 좀 쉽다. start value 라고 적어두었던 파라미터는 RDD 를 n 개로 파티셔닝 했을 때 그 각각의 파티셔닝 안의 0 번째 인덱스들 이라고 이해하면 되겠다.

그래서 아까 rdd1 은 1 개의 파티션 [1,2,3,4,5] 였었고 add function 을 이용해 각 요소들을 더해 1개의 값으로 축소한 것이다. 

In [19]:
# rdd5 = sc.parallelize([1,2,3,4,5], 3)

rdd5.fold(0, add)

15

위의 연산도 보면 

파티셔닝이 [1, [2, 3], [4, 5]] 일 때,

[0 [0, 1] [0, 2, 3], [0, 4, 5]] 이렇게 되어 연산된다고 보면 되겠다. 

start-value 인 0, 과 파티션 3 개에 0이 들어간 모습이다. 순서는 솔직히 잘 모르겠다.

결국 reduce 와의 차이는 첫 번째 값으로 zero-value 가 들어간다는 점만 차이가 있다.

zero-value 라고 꼭 0 만 넣는 것은 아니다.

In [29]:
rdd5.fold(2, add)

23

위 코드의 계산 결과를 보면 유추가 가능하다.

파티션 3 개에 2 라는 start-value 가 추가돼 15 에서 6 만큼 늘어난 21 이 아닌

zero-value = 2

(각 파티션 3 개에 추가된 2) = 3 * 2

그래서 15 + 8 = 23 이라는 결과값이 나온다.

상식적으로는 21 이 맞는 것 같은데 결과값이 23 이 나와 유추해본 결과이다.

23 이 되려면 파티셔닝이 4 개로 되는 것 아닌가..?

In [30]:
rdd5.fold(2, lambda x, y: x*y)

1920

[2 [2, 1] [2, 2, 3], [2, 4, 5]] 라고 가정하고 계산해보면

2 * 2 * 12 * 40 = 1920  이렇게 계산하나봐요..

# 3. groupby

In [42]:
rdd6 = sc.parallelize([1, 2, 2, 3, 3, 4, 5, 6, 7, 7])

res = rdd6.groupBy(lambda x: (x%2)).collect()

res

[(1, <pyspark.resultiterable.ResultIterable at 0x22d638368b0>),
 (0, <pyspark.resultiterable.ResultIterable at 0x22d63836ee0>)]

위와 같은 결과로,

나머지가 0 인 애들인 iterable 한 pyspark의 내부 클래스와

나머지가 1 인 애들인 iterable 한 pyspark의 내부 클래스가 나온다. 

iterable 한 pyspark의 내부 클래스들을 python object 로 변경해서 살펴보면 다음과 같다.

In [43]:
list(res[0][1]), list(res[1][1])

([1, 3, 3, 5, 7, 7], [2, 2, 4, 6])

In [41]:
sorted([(x, sorted(y)) for (x, y) in res])

[(0, [2, 2, 4, 6]), (1, [1, 3, 3, 5, 7, 7])]

위 코드를 보면 

x 에 해당하는 값은 각 튜플의 0 번 인덱스의 값이고, < 후에 설명할 key 에 해당하는 값>

y 에 해당하는 값들은 iterable 한 pyspark 집합들 < 후에 설명할 value 에 해당하는 값> 인데 python object 로 바꿔서 확인한 결과이다.

문법이 생소할 수도 있어 설명하면, 리스트 컴프리헨션이라는 것이고 파이썬의 장점이기도 한 코드이다.

# 4. aggregate 

개인적으로 좀 이해가 힘들더라. 좀 복잡하다.

In [47]:
rdd7 = sc.parallelize([1, 2, 3, 4])

seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))

rdd7.aggregate((0, 0), seqOp, combOp)

(10, 4)

aggregate 의 파라미터를 remind 하면 

rdd.aggregate(zero-value, seqOp, combOp) 였다. 

zero-value 는 알고 있 듯 start - value 이고,

seqOp 는 타입을 변경하는 함수이며,

__seqOp = (lambda x, y: (x[0] + y, x[1] + 1))__

combOp 는 합을 하는 함수였다.

__combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))__

연산 결과를 추적해보면 

start - value (0,0) 으로 pair 하게 파티셔닝한다.

그래서 RDD 를 [1, 2] 와 [3, 4] 로 pair 하게 파티셔닝한다.

__zero - value 는 x[0] = 0, x[1] = 0 이다__

seq 연산한 후, combOp 로 합쳐보자

[1, 2] 의 seq 의 결과는 다음과 같다.

x[0] + y, x[1] + 1 = 0 + 1, 0 + 1 이고 (1, 1) 

x[0] + y, x[1] + 1 = 1 + 2, 1 + 1 -> (3, 2) 이 된다.

첫 번째 파티션의 seq 결과 (3, 2)

[3, 4] 의 seq 의 결과는 다음과 같다.

x[0] + y, x[1] + 1 = 0 + 3, 0 + 1  이고 (3, 1) 

x[0] + y, x[1] + 1 = 3 + 4, 1 + 1 -> (7, 2) 이 된다.

두 번째 파티션의 seq 결과 (7, 2)

마지막 combOp 로 그 둘의 결과를 합치면 

(3, 2) + (7, 2) = (10, 4) 가 된다. 

코드가 뭔가 복잡하고 연산 방식이 처음엔 눈에 잘 들어오지 않고 튕겨져 나가더라.


### 정리.

위의 4 개의 코드들은 필수적으로 알아야 하는 reduction 코드들이며 익숙해져서 자유롭게 사용하게 할 수 있게끔 해야 한다고 생각한다. 

크고 복잡한 데이터들을 정제된 데이터로 축약하는 한마디로 EDA 를 하는 코드들이라 볼 수 있겠다.